In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
import os
import json
from PIL import Image
from torch.utils.data import Dataset, DataLoader, RandomSampler
from torch.nn import Linear
from tqdm import tqdm
from torchsummary import summary
import numpy as np
import random

In [20]:
if torch.cuda.is_available():
    print("Using the GPU. You are good to go!")
    device = 'cuda'
else:
    print("Using the CPU. Overall speed will be slowed down")
    device = 'cpu'

Using the GPU. You are good to go!


In [21]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
model.fc = Linear(in_features=512, out_features=4, bias=True)
model = model.to(device)

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

criterion = nn.CrossEntropyLoss()
learning_rate = 1e-3
weight_decay = 1e-4
num_epoch = 50
optimizer = optim.Adam(model.parameters(), lr=learning_rate,
                       weight_decay=weight_decay)

Using cache found in C:\Users\lizca/.cache\torch\hub\pytorch_vision_v0.10.0


In [22]:
training_data = torchvision.datasets.ImageFolder('./dataset_new/train', transform=preprocess)
test_data = torchvision.datasets.ImageFolder('./dataset_new/test', transform=preprocess)

train_loader = DataLoader(training_data, batch_size=32, sampler=RandomSampler(training_data, replacement=True, num_samples=int(len(training_data) * 0.75)))
val_loader = DataLoader(training_data, batch_size=32, sampler=RandomSampler(training_data, replacement=True, num_samples=int(len(training_data) - len(training_data) * 0.75)))
test_loader = DataLoader(training_data, batch_size=32, shuffle=True)

In [23]:
%%time
def train(model, trainloader, val_loader, num_epoch=10):  # Train the model
    print("Start training...")
    trn_loss_hist = []
    trn_acc_hist = []
    val_acc_hist = []
    model.train() 
    
    for i in range(num_epoch):
        running_loss = []
        print('-----------------Epoch = %d-----------------' % (i+1))
        for batch, label in tqdm(trainloader):
            batch = batch.to(device)
#             print(batch[0].shape)
#             transform = transforms.ToPILImage()
#             img = transform(batch[0])
#             img.show()
#             input("This is the first image")
 
            label = label.to(device)
            optimizer.zero_grad()  
            pred = model(batch)
            loss = criterion(pred, label)  
            running_loss.append(loss.item())
            loss.backward()  
            optimizer.step()  
        print("\n Epoch {} loss:{}".format(i+1, np.mean(running_loss)))

        #trn_loss_hist.append(np.mean(running_loss))
        #trn_acc_hist.append(evaluate(model, trainloader))
        print("\n Evaluate on validation set...")
        val_acc_hist.append(evaluate(model, val_loader))
    print("Done!")
    return trn_loss_hist, trn_acc_hist, val_acc_hist

def evaluate(model, loader):
    model.eval()  
    correct = 0
    with torch.no_grad():  
        for batch, label in tqdm(loader):
            batch = batch.to(device)
            label = label.to(device)
            pred = model(batch)
            correct += (torch.argmax(pred, dim=1) == label).sum().item()
        print("The model got ", str(correct), " out of ", str(len(val_loader) * 32))
        acc = correct/(len(test_loader) * 32)
        print("\n Evaluation accuracy: {}".format(acc))
        return acc


trn_loss_hist, trn_acc_hist, val_acc_hist = train(model, train_loader, val_loader, num_epoch)

print("\n Evaluate on test set")
#evaluate(model, test_loader)

Start training...
-----------------Epoch = 1-----------------


100%|██████████████████████████████████████████████████████████████████████████████████| 58/58 [00:19<00:00,  3.00it/s]



 Epoch 1 loss:0.1429873656798636

 Evaluate on validation set...


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:05<00:00,  3.60it/s]


The model got  613  out of  640

 Evaluation accuracy: 0.2455929487179487
-----------------Epoch = 2-----------------


...

The model got  615  out of  640

 Evaluation accuracy: 0.24639423076923078
-----------------Epoch = 50-----------------


100%|██████████████████████████████████████████████████████████████████████████████████| 58/58 [00:15<00:00,  3.74it/s]



 Epoch 50 loss:0.0034469060209599335

 Evaluate on validation set...


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:04<00:00,  4.06it/s]

The model got  614  out of  640

 Evaluation accuracy: 0.24599358974358973
Done!

 Evaluate on test set
Wall time: 17min 7s


In [24]:
os.makedirs('./models', exist_ok=True)
torch.save(model.state_dict(), './models/model_skiplink_{}.pth'.format("Trained_Model"))